In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-23 13:23:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-06-23 13:23:23 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-D2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [27]:
#Create Vine DF 
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [37]:
filtered1 = vine_df.filter("total_votes >=20")

In [38]:
filtered1.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [56]:
filtered2 = filtered1.withColumn("fiftyandabove", (filtered1["helpful_votes"] / filtered1["total_votes"]) >= 0.5)

In [57]:
filtered2.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|fiftyandabove|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|         true|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|         true|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|         true|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|         true|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|         true|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|         true|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|         true|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|   

In [61]:
filtered2=filtered2.filter("fiftyandabove == true")


In [62]:
filtered2.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|fiftyandabove|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|         true|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|         true|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|         true|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|         true|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|         true|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|         true|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|         true|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|   

In [85]:
filtered3 = filtered2.filter(filtered2["vine"] == "Y")
filtered3.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|fiftyandabove|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|         true|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|         true|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|         true|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|         true|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|         true|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|         true|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|         true|
|R1W3KCDHM6BMZT|          4|           37|         38|   Y|                N|   

In [86]:
filtered4 = filtered2.filter(filtered2["vine"] == "N")
filtered4.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|fiftyandabove|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|         true|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|         true|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|         true|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|         true|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|         true|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|         true|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|         true|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|   

In [90]:
review_sum_vine = filtered3.select("review_id").count()
review_sum_vine

107

In [94]:
five_star_vine = filtered3.filter("star_rating == 5").count()
five_star_vine


56

In [96]:
ratio_vine = (five_star_vine / review_sum_vine) * 100
ratio_vine

52.336448598130836

In [97]:
review_sum_nonVine  = filtered4.select("review_id").count()
review_sum_nonVine

39869

In [98]:
five_star_noneVine = filtered4.filter("star_rating == 5").count()
five_star_noneVine

21005

In [99]:
ratio_nonVine = (five_star_noneVine / review_sum_nonVine) * 100
ratio_nonVine

52.68504351751988